In [ ]:
import pybamm
import numpy as np
import os
%matplotlib inline
import matplotlib.pyplot as plt
os.chdir(pybamm.__path__[0]+'/..')

In [ ]:
model = pybamm.li_ion.SPM()

In [ ]:
# Set parameters for the model
param = model.default_parameter_values
param.process_model(model)


In [ ]:
# Create a mesh
geometry = model.default_geometry
param.process_geometry(geometry)
mesh = pybamm.Mesh(
            geometry, model.default_submesh_types, model.default_submesh_pts
        )

In [ ]:
# Discretise model over mesh
disc = pybamm.Discretisation(mesh, model.default_spatial_methods)
disc.process_model(model)

In [ ]:
# Solve the model at the given time points
solver = model.default_solver

n = 100
t_eval = np.linspace(0, 1, n)
solver.solve(model, t_eval)

In [ ]:
def evaluate_variable_over_time(exp, t, y):
    var_t0 = exp.evaluate(t[0],y[:,0])
    var_over_time = np.empty((len(var_t0),n))
    for i,_ in enumerate(solver.t):
        var_over_time[:,i] = exp.evaluate(t[i],y[:,i])
    return var_over_time


In [ ]:
variable_names = list(model.variables.keys())
fig,axs = plt.subplots(len(variable_names),figsize=(10,7))
for name, ax in zip(variable_names,axs):
    data = evaluate_variable_over_time(model.variables[name],solver.t,solver.y)
    if data.shape[0] == 1:
        ax.plot(solver.t,data.reshape(-1))
    else:
        ax.imshow(data)
    ax.set_title(name)
plt.tight_layout()
plt.show()
